In [1]:
import re
import json
import ast

import pandas as pd
from tqdm import tqdm
import numpy as np

In [11]:
df = pd.read_csv("../data/raw/all_curves.csv", engine='python')

In [15]:
df.shape

(148920, 12)

In [18]:
df['prop_y'].value_counts()

prop_y
"Seebeck coefficient"            33439
"Thermal conductivity"           24063
"ZT"                             19401
"Electrical resistivity"         18158
"Power factor"                   16058
"Electrical conductivity"        15876
"Voltage"                         6748
"magnetization_per_weight"        6150
"Discharge capacity"              5022
"magnetization_per_volume"        1220
"Carrier mobility"                1087
"Magnetization"                    889
"Hall coefficient"                 496
"Magnetic field"                   158
"Magnetic field strength (H)"      104
"magnetization_Bohr"                37
"Temperature"                       12
"Cycle number"                       2
Name: count, dtype: int64

In [12]:
df

,SID,DOI,composition,sample_id,figure_id,prop_x,prop_y,unit_x,unit_y,x,y,project_names
0,6,10.1021/am405410e,Pb1.00075Zn0.01Te1.01I0.0015,39,79,Temperature,Seebeck coefficient,K,V*K^(-1),"[299.633124,324.580713,349.895178,375.209644,3...","[-0.000072,-0.000079,-0.000089,-0.000099,-0.00...","[ThermoelectricMaterials""]"""
1,6,10.1021/am405410e,Pb1.00025Zn0.01Te1.01I0.0005,47,79,Temperature,Seebeck coefficient,K,V*K^(-1),"[299.633124,324.580713,349.528302,374.842767,3...","[-0.000135,-0.000144,-0.000156,-0.000167,-0.00...","[ThermoelectricMaterials""]"""
2,6,10.1021/am405410e,Pb1.0015Zn0.02Te1.02I0.003,64,79,Temperature,Seebeck coefficient,K,V*K^(-1),"[299.98311,324.995749,350.007985,375.020222,40...","[-0.000062,-0.000071,-0.00008,-0.00009,-0.0000...","[ThermoelectricMaterials""]"""
3,6,10.1021/am405410e,Pb1.0015Zn0.01Te1.01I0.003,69,79,Temperature,Seebeck coefficient,K,V*K^(-1),"[300,324.947589,349.895178,374.842767,399.0566...","[-0.000055,-0.000063,-0.000072,-0.000081,-0.00...","[ThermoelectricMaterials""]"""
4,6,10.1021/am405410e,Pb1.0005Zn0.01Te1.01I0.001,98,79,Temperature,Seebeck coefficient,K,V*K^(-1),"[299.633,324.948,349.895,374.843,400.157,425.1...","[-0.000109,-0.000118,-0.000128,-0.000138,-0.00...","[ThermoelectricMaterials""]"""
...,...,...,...,...,...,...,...,...,...,...,...,...
162123,46499,10.1016/j.memsci.2009.01.023,SrCo0.8Fe0.2O3,65924,46765,Temperature,Electrical conductivity,K,S*m^(-1),"[873.05,883.15,892.95,902.95,912.95,923.15,932...","[9687.9,9536.8,9397.9,9242.6,9067,8891.3,8719....","[ThermoelectricMaterials""]"""
162124,46500,10.1016/j.jpowsour.2008.12.005,Y0.7Ca0.3CrO3,65925,46766,Temperature,Electrical conductivity,K,S*m^(-1),"[772.55,822.85,872.75,922.95,972.85,1023.15,10...","[132,148.1,159.4,173.1,184,195.3,210,225.4]","[ThermoelectricMaterials""]"""
162125,46500,10.1016/j.jpowsour.2008.12.005,Y0.7Ca0.3Cr0.95Zn0.05O3,65926,46766,Temperature,Electrical conductivity,K,S*m^(-1),"[772.85,822.75,872.75,922.85,973.05,1022.85,10...","[409.8,445.7,493.6,530.3,565.1,589.5,625.7,649.8]","[ThermoelectricMaterials""]"""
162126,46500,10.1016/j.jpowsour.2008.12.005,Y0.7Ca0.3Cr0.9Zn0.1O3,65927,46766,Temperature,Electrical conductivity,K,S*m^(-1),"[772.75,822.75,872.75,922.75,972.85,1022.95,10...","[1282,1413,1552,1673,1790,1900,1997,2092]","[ThermoelectricMaterials""]"""


In [16]:
df[df['prop_y'] == 'Electrical resistivity']['DOI'].unique()

array([], dtype=object)

In [19]:
def convert_string_to_list(string_value):    
    try:        
        return eval(string_value)
    except Exception as e:
        print(string_value)
        print(e)

df['y'] = df['y'].apply(convert_string_to_list)
df['x'] = df['x'].apply(convert_string_to_list)

In [20]:
df.dtypes

SID               int64
DOI              object
composition      object
sample_id         int64
figure_id         int64
prop_x           object
prop_y           object
unit_x           object
unit_y           object
x                object
y                object
project_names    object
dtype: object

In [21]:
df = df.explode(['x', 'y']).reset_index(drop=True)

In [24]:
df

,SID,DOI,composition,sample_id,figure_id,prop_x,prop_y,unit_x,unit_y,x,y,project_names
0,6,"""10.1021/am405410e""","""Pb1.00025Zn0.02Te1.02I0.0005""",113,79,"""Temperature""","""Seebeck coefficient""","""K""","""V*K^(-1)""",299.859651,-0.000148,"[""ThermoelectricMaterials""]"
1,6,"""10.1021/am405410e""","""Pb1.00025Zn0.02Te1.02I0.0005""",113,79,"""Temperature""","""Seebeck coefficient""","""K""","""V*K^(-1)""",324.868269,-0.00016,"[""ThermoelectricMaterials""]"
2,6,"""10.1021/am405410e""","""Pb1.00025Zn0.02Te1.02I0.0005""",113,79,"""Temperature""","""Seebeck coefficient""","""K""","""V*K^(-1)""",349.87568,-0.000173,"[""ThermoelectricMaterials""]"
3,6,"""10.1021/am405410e""","""Pb1.00025Zn0.02Te1.02I0.0005""",113,79,"""Temperature""","""Seebeck coefficient""","""K""","""V*K^(-1)""",375.245424,-0.000182,"[""ThermoelectricMaterials""]"
4,6,"""10.1021/am405410e""","""Pb1.00025Zn0.02Te1.02I0.0005""",113,79,"""Temperature""","""Seebeck coefficient""","""K""","""V*K^(-1)""",399.898544,-0.000193,"[""ThermoelectricMaterials""]"
...,...,...,...,...,...,...,...,...,...,...,...,...
2767825,41688,"""10.1021/jacs.7b01851""","""PbCoO3""",46306,36159,"""Temperature""","""Electrical resistivity""","""K""","""ohm*m""",254.600219,787.234097,"[""ThermoelectricMaterials"",""CondensedMatter""]"
2767826,41688,"""10.1021/jacs.7b01851""","""PbCoO3""",46306,36159,"""Temperature""","""Electrical resistivity""","""K""","""ohm*m""",308.731028,293.448888,"[""ThermoelectricMaterials"",""CondensedMatter""]"
2767827,41688,"""10.1021/jacs.7b01851""","""PbCoO3""",46306,36159,"""Temperature""","""Electrical resistivity""","""K""","""ohm*m""",377.335315,130.882963,"[""ThermoelectricMaterials"",""CondensedMatter""]"
2767828,41688,"""10.1021/jacs.7b01851""","""PbCoO3""",46306,36159,"""Temperature""","""Electrical resistivity""","""K""","""ohm*m""",426.701612,91.419525,"[""ThermoelectricMaterials"",""CondensedMatter""]"


In [25]:
for prop, count in df['prop_x'].value_counts(normalize=False).items():
    df.loc[df['prop_x'] == prop, prop] = pd.to_numeric(df['x'])

for prop, count in df['prop_y'].value_counts(normalize=False).items():
    df.loc[df['prop_y'] == prop, prop] = pd.to_numeric(df['y'])

In [27]:
df.dtypes

SID                               int64
DOI                              object
composition                      object
sample_id                         int64
figure_id                         int64
prop_x                           object
prop_y                           object
unit_x                           object
unit_y                           object
x                                object
y                                object
project_names                    object
"Temperature"                    object
"Magnetic field strength (H)"    object
"Cycle number"                   object
"Discharge capacity"             object
"Charge capacity"                object
"Magnetic field"                 object
"Magnetization"                  object
"C rate"                         object
"Magnetic Field"                 object
"Voltage"                        object
"magnetization_per_weight"       object
"Electrical conductivity"        object
"magnetization_per_volume"       object


In [28]:
df["figure_id"] = df["figure_id"].apply(str)
df["SID"] = df["SID"].apply(str)
df["sample_id"] = df["sample_id"].apply(str)

In [30]:
df.describe(include='all')

,SID,DOI,composition,sample_id,figure_id,prop_x,prop_y,unit_x,unit_y,x,...,"""Electrical conductivity""","""magnetization_per_volume""","""Seebeck coefficient""","""Electrical resistivity""","""Carrier mobility""","""Thermal conductivity""","""ZT""","""Power factor""","""Hall coefficient""","""magnetization_Bohr"""
count,2767830,2767830,2767830,2767830,2767830,2767830,2767830,2767830,2767830,2765487,...,235871,39347,646068,445425,12671,369437,268720,234638,10956,2985
unique,10566,10566,25466,60118,41095,16,18,12,18,2209176,...,209107,35184,7130,53080,9242,319378,194278,12041,719,3
top,43147,"""10.1063/1.5023427""","""undefined""",3883,35848,"""Temperature""","""Seebeck coefficient""","""K""","""V*K^(-1)""",300,...,0,0,0,0,0,2,0,0,0,0
freq,3428,3428,438735,640,1412,2223809,646565,2223809,646565,3184,...,394,95,11664,26299,207,102,1532,4881,8588,1682


In [31]:
df.columns

Index(['SID', 'DOI', 'composition', 'sample_id', 'figure_id', 'prop_x',
       'prop_y', 'unit_x', 'unit_y', 'x', 'y', 'project_names',
       '"Temperature"', '"Magnetic field strength (H)"', '"Cycle number"',
       '"Discharge capacity"', '"Charge capacity"', '"Magnetic field"',
       '"Magnetization"', '"C rate"', '"Magnetic Field"', '"Voltage"',
       '"magnetization_per_weight"', '"Electrical conductivity"',
       '"magnetization_per_volume"', '"Seebeck coefficient"',
       '"Electrical resistivity"', '"Carrier mobility"',
       '"Thermal conductivity"', '"ZT"', '"Power factor"',
       '"Hall coefficient"', '"magnetization_Bohr"'],
      dtype='object')

In [32]:
# frequent prop
df_fp = df.groupby(['prop_x', 'prop_y']).agg({'unit_x': 'first', 'unit_y': 'first'}).reset_index()

In [34]:
df_fp

,prop_x,prop_y,unit_x,unit_y
0,"""C rate""","""Cycle number""","""-""","""-"""
1,"""C rate""","""Discharge capacity""","""-""","""A*s/kg"""
2,"""C rate""","""Voltage""","""-""","""V"""
3,"""Carrier mobility""","""Seebeck coefficient""","""m^2*V^(-1)*s^(-1)""","""V*K^(-1)"""
4,"""Charge capacity""","""Discharge capacity""","""A*s/kg""","""A*s/kg"""
5,"""Charge capacity""","""Voltage""","""A*s/kg""","""V"""
6,"""Cycle number""","""Discharge capacity""","""-""","""A*s/kg"""
7,"""Cycle number""","""Voltage""","""-""","""V"""
8,"""Discharge capacity""","""Voltage""","""A*s/kg""","""V"""
9,"""Electrical conductivity""","""Power factor""","""ohm^(-1)*m^(-1)""","""W*m^(-1)*K^(-2)"""


In [35]:
from bokeh.plotting import figure, output_file, save, show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, HoverTool, Label, Div

for index, row in df_fp.iterrows():
    prop_x = row['prop_x']
    prop_y = row['prop_y']
    unit_x = row['unit_x']
    unit_y = row['unit_y']
    label_x = f'{prop_x} ({unit_x})'
    label_y = f'{prop_y} ({unit_y})'

    selected_df = df[(df[prop_x].notnull()) & (df[prop_y].notnull())]
    
    source = ColumnDataSource(data=dict(
        x=selected_df[prop_x].values,
        y=selected_df[prop_y].values,
        SID=selected_df['SID'],
        figure_id=selected_df['figure_id'],
        sample_id=selected_df['sample_id'],
        composition=selected_df['composition']
    ))
    if prop_x == prop_y:
        continue
    if index == 2:
        break
    # continue

    p = figure(title=f'{prop_x} vs. {prop_y}', x_axis_label=label_x, y_axis_label=label_y, x_axis_type='log', y_axis_type='log')
    
    p.scatter('x', 'y', source=source, alpha=0.2)

    file_name = f'../reports/figures/html/{prop_x}-{prop_y}'
    # ツールチップの設定
    hover = HoverTool(tooltips=[('SID', '@SID'), ('Figure ID', '@figure_id'), ('Sample ID', '@sample_id'), ('composition', '@composition')])
    p.add_tools(hover)
    
    # 説明文を作成
    # caption = Div(text="<h3>この線グラフはサンプルデータを示しています</h3>")

    # レイアウトを作成
    # layout = column(caption, p)

    output_file(filename=file_name + '.html')
    # export_png(p, filename=file_name + '.png')
    # show(p)   

    save(p)

In [9]:
!pip3.10 list

Package            Version
------------------ --------
appnope            0.1.3
arrow              1.2.3
asttokens          2.2.1
backcall           0.2.0
binaryornot        0.4.4
bokeh              3.1.1
certifi            2023.5.7
chardet            5.1.0
charset-normalizer 3.1.0
click              8.1.3
comm               0.1.3
contourpy          1.0.7
cookiecutter       2.1.1
debugpy            1.6.7
decorator          5.1.1
executing          1.2.0
idna               3.4
ipykernel          6.23.1
ipython            8.14.0
jedi               0.18.2
Jinja2             3.1.2
jinja2-time        0.2.0
jupyter_client     8.2.0
jupyter_core       5.3.0
MarkupSafe         2.1.3
matplotlib-inline  0.1.6
nest-asyncio       1.5.6
numpy              1.24.3
packaging          23.1
pandas             2.0.2
parso              0.8.3
pexpect            4.8.0
pickleshare        0.7.5
Pillow             9.5.0
pip                23.0.1
platformdirs       3.5.1
prompt-toolkit     3.0.38
psutil        